In [1]:
import os
import time
import csv
import re
import codecs
import contextlib
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException

## setup area

In [2]:
read_filename = "names.csv"
write_filename = "review1.csv"

read_path =os.path.join("response", read_filename)
write_path = os.path.join("review",write_filename)

In [3]:
list_kategori = ['opsi_layanan','keunggulan','aksesbilitas','penawaran',
                'pilihan_makanan','fasilitas','suasana','tipe_pengunjung',
                'perencanaan','pembayaran']

In [4]:
# Initialize empty lists for each category
opsi_layanan = []
keunggulan = []
aksesbilitas = []
penawaran = []
pilihan_makanan = []
fasilitas = []
suasana = []
tipe_pengunjung = []
perencanaan = []
pembayaran = []

In [5]:
number = 2

## function bantuan

In [6]:
def clean_filename(filename):
    global cleaned_filename
    # Remove special characters using regular expressions
    cleaned_filename = re.sub(r'[^\w\s.-]', '', filename)

## Google maps function

In [7]:
def search(name):
    global driver , actions
    global link
    
    link = "https://www.google.com/maps/search/{} jogja".format(name)
    
    #open up chrome and stuff
    driver = webdriver.Chrome()
    driver.get(link)
    actions = ActionChains(driver)
    
    #kalau gak langsung ketemu
    
    try:
        restaurant_link = driver.find_elements(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/a')
        
        for links in restaurant_link:
            link = links.get_attribute("href")
            driver.get(link)
    except:
        pass

In [8]:
def about_button():
    try:
        about_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[3]')))
        actions.move_to_element(about_button)
        actions.click().perform()
        
    except:
        about_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]')))
        actions.move_to_element(about_button)
        actions.click().perform()

## scrape about function

In [9]:
def scrape_list(xpath):
    global points , number
    points = []
    
    try:
        list_element = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath.format(number))))
        # Extract all the points from the list
        point_elements = list_element.find_elements(By.TAG_NAME, "li")

        # Check the class of each point and append the points with the specified class
        for point_element in point_elements:
            point_class = point_element.get_attribute("class")
            if point_class == "hpLkke ":
                points.append(point_element.text)
    
    except:
        points.append('-')
        
    number +=3

In [10]:
list_kategori = ['opsi_layanan','keunggulan','aksesbilitas','penawaran',
                'pilihan_makanan','fasilitas','suasana','tipe_pengunjung',
                'perencanaan','pembayaran']

In [11]:
def scrape_about():
    global deskripsi , number
    global opsi_layanan , keunggulan , aksesbilitas
    global penawaran , pilihan_makanan , fasilitas
    global suasana , tipe_pengunjung , perencanaan
    global pembayaran
    number = 2
    xpath_deskripsi = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[{}]/p'.format(number)
    xpath_feature = '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[{}]/ul'
    deskripsi = '-'
    
    scrape_list(xpath_feature)
    opsi_layanan = points

    scrape_list(xpath_feature)
    keunggulan = points
    
    scrape_list(xpath_feature)
    aksesbilitas = points
    
    scrape_list(xpath_feature)
    penawaran = points
    
    scrape_list(xpath_feature)
    pilihan_makanan = points
    
    scrape_list(xpath_feature)
    fasilitas = points
    
    scrape_list(xpath_feature)
    suasana = points
    
    scrape_list(xpath_feature)
    tipe_pengunjung = points
    
    scrape_list(xpath_feature)
    perencanaan = points

    scrape_list(xpath_feature)
    pembayaran = points
    
    
    try:
        raw_deskripsi = WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.XPATH, xpath_deskripsi)))
        deskripsi = raw_deskripsi.text
    
    except:
        pass
    


### setup csv file

In [12]:
# Define the fieldnames for the CSV file
fieldnames = ['DESCRIPTION','SERVICE OPTION','HIGHLIGHTS' , 'ACCESSIBILITY' , 'OFFERINGS' , 'DINING OPTIONS' , 'AMENITIES' , 'ATHMOSPHERE', 'CROWD', 'PLANNING', 'PAYMENTS']

In [13]:
driver = webdriver.Chrome()
# Create the 'review' directory if it doesn't exist
os.makedirs('feature', exist_ok=True)

# Open the input file and loop through each vessel name
with open(read_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)

    total_rows = sum(1 for _ in reader)  # Get the total number of rows (excluding header row)
    csvfile.seek(0)  # Reset the file pointer to start from the beginning
    next(reader)  # Skip the header row

    progress_bar = tqdm(total=total_rows, ncols=80, desc="Scraping features")  # Create the tqdm progress bar

    for row in reader:
        clean_filename(row[0])
        NAMA = cleaned_filename
        nomor = 1
        review_count = 0

        # Generate the write file path based on the restaurant name
        write_filename = f"{NAMA}.csv"
        write_path = os.path.join("feature", write_filename)

        # Check if the file already exists and has more than 1 line
        if os.path.exists(write_path):
            with open(write_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                non_empty_lines = [line for line in lines if line.strip()]  # Filter non-empty lines
                if len(non_empty_lines) > 1:
                    progress_bar.set_postfix_str(f"{NAMA} is already scrapped. Check or delete the file if you want to rescrape it.")
                    driver.quit()  # Close the webdriver
                    progress_bar.update(1)  # Manually update the progress bar
                    continue

        try:
            search(NAMA)
        except:
            progress_bar.update(1)  # Manually update the progress bar
            continue

        try:
            about_button()
        except:
            progress_bar.update(1)  # Manually update the progress bar
            continue

        scrape_about()

        details_dict = {
            'DESCRIPTION': deskripsi,
            'SERVICE OPTION': opsi_layanan,
            'HIGHLIGHTS': keunggulan,
            'ACCESSIBILITY': aksesbilitas,
            'OFFERINGS': penawaran,
            'DINING OPTIONS': pilihan_makanan,
            'AMENITIES': fasilitas,
            'ATHMOSPHERE': suasana,
            'CROWD': tipe_pengunjung,
            'PLANNING': perencanaan,
            'PAYMENTS': pembayaran
        }

        # Determine the maximum length among all lists
        max_length = max(len(value) if isinstance(value, list) else 1 for value in details_dict.values())

        # Open the CSV file for writing with UTF-8 encoding
        with codecs.open(write_path, mode='w', encoding='utf-8', errors='ignore') as details_file:
            writer = csv.writer(details_file)
            writer.writerow(details_dict.keys())

            # Write the data to the CSV file row-wise
            for i in range(max_length):
                row_data = {}
                for key, value in details_dict.items():
                    if isinstance(value, list) and i < len(value):
                        row_data[key] = value[i]
                    elif not isinstance(value, list) and i == 0:
                        row_data[key] = value
                    else:
                        row_data[key] = ''

                writer.writerow(row_data.values())

        progress_bar.set_postfix_str(f"Scraped features for {NAMA}")  # Update the progress bar description
        progress_bar.update(1)  # Manually update the progress bar

    progress_bar.close()  # Close the progress bar

print('Done!')
driver.quit()  # Close the webdriver


Scraping features:   1%| | 6/551 [01:44<2:56:00, 19.38s/it, SENJA croissant sand

starting scraping for:Angkringan Kopi Joss Tugu Mbak lina


Scraping features:   2%| | 9/551 [02:17<2:12:32, 14.67s/it, Angkringan wulan nda

starting scraping for:Kopi Joss Lek No


Scraping features:   2%| | 10/551 [02:28<2:00:33, 13.37s/it, Scraped features fo

...

starting scraping for:Karaoke Edy


Scraping features: 100%|▉| 550/551 [2:27:57<00:17, 17.21s/it, Scraped features f

starting scraping for:Ralls  Dwy


Scraping features: 100%|█| 551/551 [2:28:12<00:00, 16.14s/it, Scraped features f


Done!


In [ ]:
#Test cell

test ="'D'coff jogja"
search(test)
about_button()
scrape_about()

In [ ]:
# def scrape_list():
#     delay = 5
#     #opsi layanan
#     try:
#         list_element = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[2]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 opsi_layanan.append(point_element.text)
    
#     except:
#         opsi_layanan.append('-')
        

#     #keunggulan
#     try:
#         list_element = WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[5]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 keunggulan.append(point_element.text)
    
#     except:
#         keunggulan.append('-')
        
#     #aksesbilitas
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[8]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 aksesbilitas.append(point_element.text)
    
#     except:
#         aksesbilitas.append('-')
        
#     #penawaran
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[11]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 penawaran.append(point_element.text)
    
#     except:
#         penawaran.append('-')
        
#     #pilihan makanan
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[13]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 piliihan_makanan.append(point_element.text)
    
#     except:
#         pilihan_makanan.append('-')
        
#     #fasilitas
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[15]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 fasilitas.append(point_element.text)
    
#     except:
#         fasilitas.append('-')
        
#     #suasana
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[18]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 suasana.append(point_element.text)
    
#     except:
#         suasana.append('-')
        
#     #tipe pengunjung
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[21]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 tipe_pengunjung.append(point_element.text)
    
#     except:
#         tipe_pengunjung.append('-')
        
#     #perencanaan
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[24]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 perencanaan.append(point_element.text)
    
#     except:
#         perencanaan.append('-')
        
#     #pembayaran
#     try:
#         list_element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[27]/ul')))
        
#         # Extract all the points from the list
#         point_elements = list_element.find_elements(By.TAG_NAME, "li")

#         # Check the class of each point and append the points with the specified class
#         for point_element in point_elements:
#             point_class = point_element.get_attribute("class")
#             if point_class == "hpLkke ":
#                 pembayaran.append(point_element.text)
    
#     except:
#         pembayaran.append('-')
        
    